In [ ]:
import os
import pandas as pd
from trials.Utilities import write_table_html

In [ ]:

base_path = 'graphs'
trials = os.listdir(base_path)

def append_epoch(source: pd.DataFrame, epoch: pd.DataFrame) -> pd.DataFrame:
    return source.append({
        'epoch_num': epoch_num,
        'parameters': epoch['parameters'],
        'train_time': epoch['train_time'],
        'test_accuracy': epoch['test_accuracy'],
        'valid_accuracy': epoch['valid_accuracy'],
        'total_accuracy': epoch['total_accuracy'],
    }, ignore_index=True)

for trial_name in trials:
    trial_path = os.path.join(base_path, trial_name)
    epochs = os.listdir(trial_path)

    med = pd.DataFrame()
    mean = pd.DataFrame()
    std = pd.DataFrame()
    min = pd.DataFrame()
    max = pd.DataFrame()

    for epoch_name in epochs:
        epoch_path = os.path.join(trial_path, epoch_name)
        if not os.path.isdir(epoch_path):
            continue

        epoch_num = int(epoch_name.replace('Epoch ', ''))

        #all = pd.read_csv(os.path.join(epoch_path, 'All.csv'))
        #best = pd.read_csv(os.path.join(epoch_path, 'Best.csv'))
        pop = pd.read_csv(os.path.join(epoch_path, 'Population.csv'))

        target = pop # for quick switching during testing
        med = append_epoch(med, target.median())
        mean = append_epoch(mean, target.mean())
        std = append_epoch(std, target.std())
        min = append_epoch(min, target.min())
        max = append_epoch(max, target.max())

    med = med.set_index(keys=['epoch_num']).sort_index()
    mean = mean.set_index(keys=['epoch_num']).sort_index()
    std = std.set_index(keys=['epoch_num']).sort_index()
    min = min.set_index(keys=['epoch_num']).sort_index()
    max = max.set_index(keys=['epoch_num']).sort_index()

    def write(df: pd.DataFrame, name: str, precision: int = 5):
        df = df.style.set_caption(f'{name} Values for {trial_name}').format(precision=precision)
        write_table_html(df, trial_path, f'{name}.html', over_write=True)

    write(med, 'Median')
    write(mean, 'Mean')
    write(std, 'Std. Deviation')
    write(min, 'Minimum')
    write(max, 'Maximum')

